In [1]:
#importing the libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import string
import re
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import sys
import string
import gensim
from gensim.models import KeyedVectors
from collections import  Counter
import spacy
from operator import itemgetter
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from nltk.tokenize import word_tokenize
import gensim
from gensim import corpora
import pyLDAvis
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import plotly.express as px

In [2]:
#Reading the data
data = pd.read_csv("Comments.csv")
data.head(5)

,COMMENT
0,Realizing that they are in the power supply bu...
1,"Futuri has grown employment by a huge amount, ..."
2,assistance for DBO's
3,The market is shifting and we aren't adapting....
4,Quicker growth and store openings so people wi...


In [3]:
#Removing the punctuations
def remove_punctuation(text):
    
    """ 
    input: takes original text string 

    output:
    removes punctuations in the text 

    """
    
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

data['COMMENT'] = data['COMMENT'].apply(lambda x:remove_punctuation(x))
data['COMMENT']

0        Realizing that they are in the power supply bu...
1        Futuri has grown employment by a huge amount b...
2                                      assistance for DBOs
3        The market is shifting and we arent adapting C...
4        Quicker growth and store openings so people wi...
                               ...                        
98584                           Growing at the right speed
98585    I like that we are not doing what we have alwa...
98586    I love everything about the direction REMAX 1 ...
98587    Expanding and growing Its great to be apart of...
98588    I like that our team of workers at wesco get a...
Name: COMMENT, Length: 98589, dtype: object

In [4]:
#Lowering the lines
data['COMMENT'] = data['COMMENT'].apply(lambda x: x.lower())
data['COMMENT']

0        realizing that they are in the power supply bu...
1        futuri has grown employment by a huge amount b...
2                                      assistance for dbos
3        the market is shifting and we arent adapting c...
4        quicker growth and store openings so people wi...
                               ...                        
98584                           growing at the right speed
98585    i like that we are not doing what we have alwa...
98586    i love everything about the direction remax 1 ...
98587    expanding and growing its great to be apart of...
98588    i like that our team of workers at wesco get a...
Name: COMMENT, Length: 98589, dtype: object

In [5]:
#Removing the Stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
stopwords[0:10]


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rutvathaker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [6]:
#Data after removing Stopwords
data['COMMENT'] = data['COMMENT'].apply(lambda x:' '.join([i for i in x.split() if i not in stopwords]))
data['COMMENT']

0        realizing power supply business generator truc...
1        futuri grown employment huge amount doesnt see...
2                                          assistance dbos
3        market shifting arent adapting customers want ...
4        quicker growth store openings people within or...
                               ...                        
98584                                  growing right speed
98585    like always done like idea copresidency like h...
98586    love everything direction remax 1 moving feel ...
98587                        expanding growing great apart
98588    like team workers wesco get along theres littl...
Name: COMMENT, Length: 98589, dtype: object

In [7]:
#Lemmatizing the Data
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

word_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
    
    """ 
    input: takes original text string 

    output:
    lemmatized text

    """
    
    lemm_word=" ".join([word_lemmatizer.lemmatize(word) for word in text.split()])
    return lemm_word
data['COMMENT']=data['COMMENT'].apply(lambda x: lemmatizer(x))
data['COMMENT']


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/rutvathaker/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/rutvathaker/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0        realizing power supply business generator truc...
1        futuri grown employment huge amount doesnt see...
2                                          assistance dbos
3        market shifting arent adapting customer want s...
4        quicker growth store opening people within org...
                               ...                        
98584                                  growing right speed
98585    like always done like idea copresidency like h...
98586    love everything direction remax 1 moving feel ...
98587                        expanding growing great apart
98588    like team worker wesco get along there little ...
Name: COMMENT, Length: 98589, dtype: object

In [8]:
#Tokenizing the data
def tokenization(text):
    tokens=re.findall("[\w']+", text)
    return tokens
data['COMMENT_tokenized']= data['COMMENT'].apply(lambda x: tokenization(x))
data['COMMENT_tokenized']

<>:3: DeprecationWarning: invalid escape sequence \w
<>:3: DeprecationWarning: invalid escape sequence \w
/tmp/ipykernel_4241/3620587533.py:3: DeprecationWarning: invalid escape sequence \w
  tokens=re.findall("[\w']+", text)


0        [realizing, power, supply, business, generator...
1        [futuri, grown, employment, huge, amount, does...
2                                       [assistance, dbos]
3        [market, shifting, arent, adapting, customer, ...
4        [quicker, growth, store, opening, people, with...
                               ...                        
98584                              [growing, right, speed]
98585    [like, always, done, like, idea, copresidency,...
98586    [love, everything, direction, remax, 1, moving...
98587                   [expanding, growing, great, apart]
98588    [like, team, worker, wesco, get, along, there,...
Name: COMMENT_tokenized, Length: 98589, dtype: object

In [9]:
#Finding sentence Length after lemmatizing in the same data
data['Sentence_length'] = data["COMMENT"].apply(lambda x: len(x.split()))
data['Sentence_length'].sort_values()

49220       0
59529       0
58252       0
30599       0
7324        0
         ... 
7488      518
54458     522
44047     632
18190     746
63437    1062
Name: Sentence_length, Length: 98589, dtype: int64

In [14]:
#Creating dictionary for id2word and creating corpus for the document term matrix for the word and its count
id2word = corpora.Dictionary(data['COMMENT_tokenized'])


corpus = []
for text in data['COMMENT_tokenized']:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[15:30])

[[(14, 1), (53, 1), (99, 1), (133, 1), (134, 1), (135, 2), (136, 1), (137, 1), (138, 1), (139, 1), (140, 1), (141, 1), (142, 1)], [(64, 1), (143, 1), (144, 1), (145, 1), (146, 1), (147, 1), (148, 1), (149, 1), (150, 1), (151, 1)], [(152, 1)], [(64, 1), (127, 1), (153, 1)], [(48, 1), (51, 1), (53, 1), (104, 1), (154, 1), (155, 1), (156, 1), (157, 1), (158, 1), (159, 1), (160, 1), (161, 1), (162, 1), (163, 1), (164, 1)], [(0, 1), (165, 1), (166, 1)], [(167, 1), (168, 1), (169, 1), (170, 1)], [(171, 1)], [(0, 3), (9, 2), (28, 2), (63, 1), (86, 1), (104, 2), (107, 1), (114, 3), (135, 2), (137, 1), (151, 1), (152, 1), (172, 1), (173, 2), (174, 1), (175, 1), (176, 1), (177, 1), (178, 1), (179, 1), (180, 1), (181, 1), (182, 2), (183, 1), (184, 1), (185, 3), (186, 1), (187, 1), (188, 1), (189, 1), (190, 1), (191, 3), (192, 1), (193, 1), (194, 2), (195, 1), (196, 1), (197, 1), (198, 2), (199, 1), (200, 1), (201, 2), (202, 1), (203, 1), (204, 1), (205, 1), (206, 1), (207, 1), (208, 1), (209, 1),

In [13]:
id2word.token2id

{'business': 0,
 'generator': 1,
 'power': 2,
 'realizing': 3,
 'repair': 4,
 'supply': 5,
 'truck': 6,
 'adjustment': 7,
 'amount': 8,
 'company': 9,
 'costofliving': 10,
 'doesnt': 11,
 'dont': 12,
 'eager': 13,
 'employee': 14,
 'employment': 15,
 'everyone': 16,
 'existing': 17,
 'futuri': 18,
 'get': 19,
 'give': 20,
 'grown': 21,
 'high': 22,
 'hire': 23,
 'huge': 24,
 'key': 25,
 'le': 26,
 'least': 27,
 'money': 28,
 'overhire': 29,
 'position': 30,
 'raise': 31,
 'regular': 32,
 'seem': 33,
 'strategically': 34,
 'swing': 35,
 'tight': 36,
 'turnover': 37,
 'wildly': 38,
 'assistance': 39,
 'dbos': 40,
 'adapting': 41,
 'arent': 42,
 'assessment': 43,
 'better': 44,
 'budget': 45,
 'cheaper': 46,
 'continue': 47,
 'customer': 48,
 'limited': 49,
 'longer': 50,
 'market': 51,
 'may': 52,
 'need': 53,
 'say': 54,
 'shifting': 55,
 'shorter': 56,
 'smaller': 57,
 'solution': 58,
 'true': 59,
 'want': 60,
 'competing': 61,
 'excited': 62,
 'focused': 63,
 'growth': 64,
 'looking':

In [15]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=corpus, id2word=id2word, num_topics=12, random_state=100,
                chunksize=1000, passes=50,iterations=100)

In [40]:
lda_model.print_topics()

[(0,
  '0.097*"like" + 0.049*"better" + 0.034*"love" + 0.027*"see" + 0.022*"u" + 0.021*"help" + 0.021*"constantly" + 0.021*"employee" + 0.021*"job" + 0.019*"work"'),
 (1,
  '0.049*"community" + 0.047*"student" + 0.038*"need" + 0.036*"best" + 0.023*"school" + 0.018*"support" + 0.018*"serve" + 0.018*"program" + 0.017*"world" + 0.016*"education"'),
 (2,
  '0.078*"care" + 0.063*"patient" + 0.052*"employee" + 0.030*"staff" + 0.021*"quality" + 0.020*"pay" + 0.018*"health" + 0.017*"hospital" + 0.015*"high" + 0.015*"benefit"'),
 (3,
  '0.107*"customer" + 0.079*"product" + 0.070*"service" + 0.066*"positive" + 0.060*"focus" + 0.019*"need" + 0.017*"focusing" + 0.015*"security" + 0.015*"changing" + 0.014*"everything"'),
 (4,
  '0.019*"new" + 0.015*"change" + 0.013*"management" + 0.011*"team" + 0.011*"need" + 0.010*"department" + 0.009*"work" + 0.009*"leadership" + 0.009*"decision" + 0.008*"process"'),
 (5,
  '0.154*"growing" + 0.072*"new" + 0.059*"company" + 0.054*"grow" + 0.046*"market" + 0.044*"

In [43]:
# Visualizing the topics
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

/home/rutvathaker/.local/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.087457  0.050537       1        1  22.248545
9     -0.270110  0.027956       2        1  21.532925
8     -0.107828  0.041921       3        1   7.224877
2      0.068801  0.190586       4        1   7.066050
11    -0.188035 -0.066536       5        1   7.042837
0     -0.220243  0.023799       6        1   6.940989
1      0.122899  0.194605       7        1   5.413196
7      0.212101  0.049416       8        1   4.963816
10     0.068687 -0.206924       9        1   4.819077
3      0.175584  0.163173      10        1   4.506260
5      0.103433 -0.240907      11        1   4.215246
6      0.122168 -0.227626      12        1   4.026183, topic_info=            Term          Freq         Total Category  logprob  loglift
64        growth  10497.000000  10497.000000  Default  30.0000  30.0000
192         like  20295.000000  20295.000000  Default  29.0000  29.0000
137      growing   8621.000000   8621.000000  Default  28.0000  28.0000
239    direction   9912.000000   9912.000000  Default  27.0000  27.0000
285          new  13499.000000  13499.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
370          lot    799.545065   4214.392972  Topic12  -4.2036   1.5501
285          new   1187.921743  13499.947081  Topic12  -3.8076   0.7819
1522       drive    356.129537    645.958464  Topic12  -5.0123   2.6169
357   competitor    339.043256    698.873535  Topic12  -5.0615   2.4890
2088    constant    311.777036    614.035211  Topic12  -5.1453   2.5346

[632 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1457      1  0.532745  ability
1457     10  0.466274  ability
1013      1  0.320343     able
1013      2  0.277741     able
1013      6  0.313150     able
...     ...       ...      ...
151       1  0.156891     year
151       2  0.731046     year
151       5  0.111906     year
1404      7  0.998465      yes
658       2  0.997656  younger

[828 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 10, 9, 3, 12, 1, 2, 8, 11, 4, 6, 7])

/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [41]:
# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=data['COMMENT_tokenized'], dictionary=id2word , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5583517121538061


In [21]:
#Defining the compute_coherence_values function
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """

    Parameters:
    
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=data['COMMENT_tokenized'], dictionary=id2word, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [22]:
#List of LDA topic models and Coherence values corresponding to the LDA model with respective number of topics
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data['COMMENT_tokenized'], start=2, limit=50, step=1)

/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/lib/python3/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [30]:
# Graph for coherence values and number of topics
limit=50; start=2; step=1;
x = range(start, limit, step)
fig=px.line(x=x, xaxis_title='Num_Topics' y=coherence_values, yaxis_title='Coherence_Values')

fig.show()# Print the coherence scores+

In [62]:
import pickle
pickle.dump(lda_model,open('lda_model.pkl','wb'))
# Load model back again
pickle.load(open('lda_model.pkl','rb'))

In [76]:
#defining the text and geeting topics and its score from the text 
document = '''i like my job'''
document = id2word.doc2bow(document.split())
lda_model.get_document_topics(document)

[(0, 0.69446945),
 (1, 0.027775373),
 (2, 0.027775373),
 (3, 0.027775763),
 (4, 0.027775373),
 (5, 0.027775373),
 (6, 0.027775373),
 (7, 0.027775373),
 (8, 0.027775645),
 (9, 0.027775912),
 (10, 0.027775373),
 (11, 0.027775604)]

In [77]:
#defining the text and geeting topics and its score from the text 
document = '''doctors take care of the patients'''
document = id2word.doc2bow(document.split())
lda_model.get_document_topics(document)

[(0, 0.027778106),
 (1, 0.027778106),
 (2, 0.6944394),
 (3, 0.027778106),
 (4, 0.027778732),
 (5, 0.027778106),
 (6, 0.027778106),
 (7, 0.027778106),
 (8, 0.027778106),
 (9, 0.027778942),
 (10, 0.027778106),
 (11, 0.027778106)]

In [78]:
#defining the text and geeting topics and its score from the text 
document = '''company culture is good'''
document = id2word.doc2bow(document.split())
lda_model.get_document_topics(document)

[(0, 0.020833638),
 (1, 0.02083347),
 (2, 0.02083347),
 (3, 0.02083347),
 (4, 0.02083347),
 (5, 0.020834086),
 (6, 0.020833781),
 (7, 0.02083347),
 (8, 0.38118875),
 (9, 0.020833835),
 (10, 0.02083347),
 (11, 0.4104751)]

In [126]:
#Giving input for the text and forming the dictionary for the topics and its coherence values and printing the score above 0.50
def get_inference():
    text = input("Enter: ")
    document = id2word.doc2bow(text.split())
    score_dic = dict(lda_model.get_document_topics(document))
    score = [i for i in score_dic.values() if i >0.5]
    return [(i,j) for i,j in score_dic.items() if j in score]

In [127]:
get_inference()

Enter: i love my job


[(0, 0.6944698)]